In [ ]:
# Import necessary libraries
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import matplotlib.pylab as plt
import seaborn as sns

from IPython.display import Image
import lightgbm as lgb
import pandas as pd
import matplotlib.pyplot as plt

import pickle
from sklearn.metrics import mean_squared_error

In [ ]:
# Load the data from CSV files
sales = pd.read_csv(r'C:\Users\aibar\Downloads\sales_train.csv')
cal = pd.read_csv(r'C:\Users\aibar\Downloads\calendar.csv')
cal_events = pd.read_csv(r'C:\Users\aibar\Downloads\calendar_events.csv')
prices = pd.read_csv(r'C:\Users\aibar\Downloads\items_weekly_sell_prices.csv')

In [ ]:
# Group events by date
grouped = cal_events.groupby('date')

# Create empty lists to store event_name and event_type
event_names = []
event_types = []

# Iterate through groups and collect events
for date, group in grouped:
    event_names.append(list(group['event_name']))
    event_types.append(list(group['event_type']))

# Create a new dataframe with the desired format
new_cal_events = {'date': grouped.groups.keys(),
            'event_name_1': [names[0] for names in event_names],
            'event_type_1': [types[0] for types in event_types],
            'event_name_2': [names[1] if len(names) > 1 else None for names in event_names],
            'event_type_2': [types[1] if len(types) > 1 else None for types in event_types]}

new_cal_events = pd.DataFrame(new_cal_events)

# Reorder columns
new_cal_events = new_cal_events[['date', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']]

# Display the transformed dataframe
#print(new_cal_events.tail())

print(new_cal_events.tail())
calendar = cal.merge(new_cal_events, on = 'date', how = 'left')
calendar['date'] = pd.to_datetime(calendar['date'])
from datetime import datetime

# Extract weekday (e.g., 'Monday' for 1, 'Tuesday' for 2, etc.)
calendar['weekday'] = calendar['date'].dt.strftime('%A')

# Extract month (e.g., 'January' for 1, 'February' for 2, etc.)
calendar['month'] = calendar['date'].dt.month

# Extract year
calendar['year'] = calendar['date'].dt.year

def map_weekday_to_wday(weekday):
    weekdays = ['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    return weekdays.index(weekday) + 1

# Apply the mapping function to create the 'wday' column
calendar['wday'] = calendar['weekday'].apply(map_weekday_to_wday)

calendar = calendar[['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']]
# here i am converting NaN value to Not_event
calendar.fillna('No_event',inplace=True)


           date    event_name_1 event_type_1  event_name_2 event_type_2
157  2016-05-08    Mother's day     Cultural          None         None
158  2016-05-30     MemorialDay     National          None         None
159  2016-06-02  NBAFinalsStart     Sporting          None         None
160  2016-06-07  Ramadan starts    Religious          None         None
161  2016-06-19    Father's day     Cultural  NBAFinalsEnd     Sporting


In [ ]:
#Add zero sales for the remaining days 1542-1549 for test data
for d in range(1542,1549):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

In [ ]:
#9 #Downcasting all the dataframes in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

In [ ]:
#Add zero sales for the remaining days 1542-1549 for test data
for d in range(1542,1549):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

In [ ]:
# Melt the sales data to have a columnar format
df_sales=pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold')

In [ ]:
# Perform garbage collection to free up memory
import gc
gc.collect()

0

In [ ]:
# Merge the sales data with the calendar data based on the 'd' column
df_final= pd.merge(df_sales, calendar, on='d', how='left')

In [ ]:
# Perform garbage collection to free up memory
del cal
del cal_events
del date
del event_names
del event_types
del group
del grouped
del new_cal_events
gc.collect()

0

In [ ]:
# Merge the df_final DataFrame with the prices DataFrame based on 'store_id', 'item_id', and 'wm_yr_wk'

df_final= pd.merge(df_final, prices, on=['store_id','item_id','wm_yr_wk'], how='left')

In [ ]:
# Fill NaN values in the 'sell_price' column with the mean sell price for each store, item, and day

df_final['sell_price'].fillna(df_final.groupby(['store_id','item_id','d'])['sell_price'].transform('mean'),inplace=True)

In [ ]:
# Fill any remaining NaN values in the 'sell_price' column with 0
df_final['sell_price'].fillna(0, inplace=True)


In [ ]:
# Calculate the revenue by multiplying 'sold' and 'sell_price' columns

df_final['revenue'] = df_final['sold'] * df_final['sell_price']

In [ ]:
# Convert 'revenue' column to integer data type
df_final['revenue'] = df_final['revenue'].astype(int)


In [ ]:
# Dropping columns with more than 50% missing values as imputing them will not be a good option
columns_to_drop = ['sold','wm_yr_wk', 'weekday', 'wday', 'month', 'year','item_id', 'dept_id', 'cat_id', 'state_id','event_name_1', 'event_type_1','event_name_2', 'event_type_2']

# Use the drop method to remove the specified columns
df_final = df_final.drop(columns=columns_to_drop)

In [ ]:
# label encoding
df_final.d = df_final['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
cols = df_final.dtypes.index.tolist()
types = df_final.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df_final[cols[i]] = df_final[cols[i]].cat.codes

In [ ]:
df_final.head()

,id,store_id,d,date,sell_price,revenue
0,14370,0,1,2011-01-29,0.0,0
1,14380,0,1,2011-01-29,0.0,0
2,14390,0,1,2011-01-29,0.0,0
3,14400,0,1,2011-01-29,0.0,0
4,14410,0,1,2011-01-29,0.0,0


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import itertools
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [ ]:

def load_and_preprocess_data(data):
    # Drop unnecessary columns and set the date as the index
    data = data.groupby(['date', 'id', 'store_id', 'd'])['revenue'].sum().reset_index()
    data = data.drop(columns=['id', 'store_id', 'd'])
    data.set_index('date', inplace=True)

    # Group data by date and sum the revenue
    data = data.groupby(data.index).sum()
    data.index = pd.to_datetime(data.index)

    # Convert 'revenue' column to numeric type
    data['revenue'] = pd.to_numeric(data['revenue'], errors='coerce')

    return data


In [ ]:

def test_stationarity(data):
    # Perform Augmented Dickey-Fuller Test
    dftest = adfuller(data, autolag='AIC')
    print("1. ADF:", dftest[0])
    print("2. P-Value:", dftest[1])
    print("3. Num of Lags :",dftest[2])
    print("4. Num of observations used for ADF Regression and critical values Calculation :", dftest[3])
    print("5. Critical Values :")
    for key, val in dftest[4].items():
        print("\t", key, ": ", val)


In [ ]:

def difference_data(data):
    # Calculate the difference between consecutive revenue values
    data['revenue'] = data['revenue'] - data['revenue'].shift(1)
    data = data.dropna()
    return data


In [ ]:

def auto_arima_model(data):
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

    best_aic = float("inf")
    best_params = None
    best_seasonal_params = None

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                model = sm.tsa.statespace.SARIMAX(data,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)
                results = model.fit()
                aic = results.aic
                if aic < best_aic:
                    best_aic = aic
                    best_params = param
                    best_seasonal_params = param_seasonal
            except:
                continue

    return best_params, best_seasonal_params


In [ ]:

# Load and preprocess data

total_revenue_df = load_and_preprocess_data(df_final)

# Test stationarity
test_stationarity(total_revenue_df['revenue'])

# Difference the data
total_revenue_df = difference_data(total_revenue_df)


In [ ]:

 # Perform automated ARIMA modeling
best_params, best_seasonal_params = auto_arima_model(total_revenue_df)
print("Best ARIMA Parameters (p, d, q):", best_params)
print("Best Seasonal Parameters (P, D, Q, S):", best_seasonal_params)


In [ ]:

# Split the data into training and validation sets
training_data, validation_data = train_test_split(total_revenue_df, test_size=0.2, shuffle=False)



In [ ]:

# Fit the best ARIMA model
model = sm.tsa.statespace.SARIMAX(training_data,
                                    order=best_params,
                                    seasonal_order=best_seasonal_params,
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)
results = model.fit()


In [ ]:

# Make predictions on the validation set
forecast = results.get_forecast(steps=len(validation_data))
forecasted_revenue = forecast.predicted_mean


In [ ]:

# Calculate RMSE on the validation set
rmse_value = np.sqrt(mean_squared_error(valid_data['revenue'], forecasted_revenue))
print("Validation RMSE:", rmse_value)


In [ ]:

# Forecast sales for the next 7 days
forecast = results.forecast(steps=7)


In [ ]:
forecast

2014-06-20    2239.969976
2014-06-21    7927.344333
2014-06-22    5399.822257
2014-06-23   -4764.303182
2014-06-24    -786.323513
2014-06-25   -5865.924878
2014-06-26   -3327.795587
Freq: D, Name: predicted_mean, dtype: float64